In [1]:
# importing necessary libraries
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from itertools import combinations
import re
import requests

# -------------------- PARAMETERS --------------------

In [2]:
# name of the JSON file
filename = 'mtg_scryfall_data_02_2023'

In [3]:
# output file name
result_file_name = "mtg_dataset_new"

In [4]:
# language to be allowed 
allowed_lang = "en"

In [5]:
# rarity to be allowed
allowed_rarity = ['common','uncommon','rare','mythic']

In [6]:
# oldest release date to be allowed
oldest_allowed_date = "2000-01-01"

Static parameters were listed below:

In [7]:
colors = ['B','G','R','U','W']

In [8]:
response = requests.get('https://api.scryfall.com/catalog/keyword-actions')
keyword_actions = (response.json())['data']

In [9]:
response = requests.get('https://api.scryfall.com/catalog/keyword-abilities')
keyword_abilities = (response.json())['data']

In [10]:
custom_categories = [
    'counterspell', 
    'manarock', 
    'manadork', 
    'point_removal', 
    'exile',
    'boardwipe', 
    'land_ramp', 
    'nonland_tutor', 
    'card_draw', 
    'burn', 
    'discard', 
    'etb', 
    'die_trigger', 
    'attack_trigger',
    'tokens',
    'creature_tokens', 
    'treasure_tokens',
    'extra_turn', 
    'plus1_counters', 
    'graveyard_hate', 
    'free_spell', 
    'bounce_spell', 
    'sac_outlet', 
    'sac_payoff', 
    'cant_counter', 
    'tax',
    'graveyard_to_hand', 
    'reanimation', 
    'cast_graveyard', 
    'lord', 
    'upkeep_trigger', 
    'endstep_trigger', 
    'combat_trigger', 
    'life_gain',  
    'cost_reduction', 
    'mana_multipliers', 
    'card_selection', 
    'cast_trigger', 
    'gain_control', 
    'evasion', 
    'copy', 
    'trigger_multiplier', 
    'damage_multipliers', 
    'instant_speed_enabler', 
    'doublers', 
    'blinker', 
    'play_toplibrary', 
    'life_lose', 
    'disenchant', 
    'animator', 
    'wish', 
    'cheatinto_play', 
    'ritual', 
    'no_maximum', 
    'wheel', 
    'extra_combat', 
    'pillow_fort', 
    'land_destruction', 
    'win_game', 
    'lose_game',
    'poison'
]

# -------------------- FUNCTIONS --------------------

In [11]:
def check_if_should_be_skipped(data_row):
    if data_row['lang'] != allowed_lang:
        return True
    
    if data_row['rarity'] not in allowed_rarity:
        return True
    
    if data_row['layout'] == 'planar':
        return True
    
    if data_row['layout'] == 'token':
        return True
    
    if data_row['layout'] == 'scheme':
        return True
    
    release_date = datetime.strptime(data_row['released_at'], '%Y-%m-%d')
    allowed_date = datetime.strptime(oldest_allowed_date, '%Y-%m-%d')
    if release_date < allowed_date:
        return True
    
    if len(data_row['multiverse_ids']) == 0:
        return True
    
    if "land" in str(data_row['type_line']).lower():
        return True
    
    if "conspiracy" in str(data_row['type_line']).lower():
        return True
    
    if "silver" in str(data_row['border_color']).lower():
        return True
    
    if "paper" not in str(data_row['games']).lower():
        return True
    
    if "funny" in str(data_row['set_type']).lower():
        return True
    
    return False

In [12]:
def data_nullable(data, name):
    has_column = data.get(name, False) != False
    if has_column:
        return data[name]
    else:
        is_dfc = data.get('card_faces', False) != False
        if is_dfc:
            for face in data['card_faces']:
                face_data = json.loads(json.dumps(face, indent = 4))
                face_has_column = face_data.get(name, False) != False
                if face_has_column:
                    return face_data[name]
            return None
        else:
            return None

In [13]:
def get_basic_columns(data, new_row):
    new_row['name'] = data['name']
    new_row['set'] = data['set']
    dual_faced_card = data.get('card_faces', False) != False
    if dual_faced_card:
        new_row['dfc'] = int(len(data['card_faces']) > 0)
    else:
        new_row['dfc'] = 0
    new_row['layout'] = data['layout']
    new_row['flavor_text'] = int(data.get('flavor_text', False) != False)
    
    return new_row

In [14]:
def get_types_columns(data, new_row):
    type_line = str(data['type_line']).lower()
    
    new_row['type_creature'] = int("creature" in type_line)
    new_row['type_artifact'] = int("artifact" in type_line)
    new_row['type_enchantment'] = int("enchantment" in type_line)
    new_row['type_planeswalker'] = int("planeswalker" in type_line)
    new_row['type_instant'] = int("instant" in type_line)
    new_row['type_sorcery'] = int("sorcery" in type_line)
    
    new_row['type_legendary'] = int("legendary" in type_line)
    new_row['type_snow'] = int("snow" in type_line)
    
    new_row['total_types'] = new_row['type_creature'] + new_row['type_artifact'] + new_row['type_enchantment'] + new_row['type_planeswalker'] + new_row['type_instant'] + new_row['type_sorcery'] + new_row['type_legendary'] + new_row['type_snow']
    
    return new_row

In [15]:
def get_mana_columns(data, new_row):
    new_row['cmc'] = data['cmc']
    new_row['X_spell'] = int("{X}" in str(data_nullable(data, 'mana_cost')))
    
    new_row['colorless'] = int(len(data_nullable(data, 'colors')) == 0)
    new_row['monocolored'] = int(len(data_nullable(data, 'colors')) == 1)
    new_row['two_colors'] = int(len(data_nullable(data, 'colors')) == 2)
    new_row['three_colors'] = int(len(data_nullable(data, 'colors')) == 3)
    new_row['four_colors'] = int(len(data_nullable(data, 'colors')) == 4)
    new_row['five_colors'] = int(len(data_nullable(data, 'colors')) == 5)
    new_row['multicolored'] = int(len(data_nullable(data, 'colors')) >= 2)
    
    for color in colors:
        new_row['devotion_' + color] = str(data_nullable(data, 'mana_cost')).count('{' + color + '}')
    new_row['devotion_Colorless'] = str(data_nullable(data, 'mana_cost')).count('{C}')
    new_row['manasymbols'] = new_row['devotion_B'] + new_row['devotion_G'] + new_row['devotion_R'] + new_row['devotion_U'] + new_row['devotion_W'] + new_row['devotion_Colorless']
    
    for color in colors:
        new_row['colors_' + color] = int(color in data_nullable(data, 'colors'))
    
    for num_of_colors in range(2, 6):
        colors_selected = list(combinations(colors, num_of_colors))
        for col_s in colors_selected:
            col_combo = ''.join(col_s)
            new_row['colors_' + col_combo] = 1
            for c in col_s:
                if new_row['colors_' + c] == 0:
                    new_row['colors_' + col_combo] = 0
                    break
    
    
    colors_multiclass = ""
    if new_row['colorless'] == 1:
        colors_multiclass = "Colorless"
    else:
        for color in colors:
            if new_row['colors_' + color] == 1:
                colors_multiclass = colors_multiclass + color
    new_row['colors_multiclass'] = colors_multiclass
    
    
    return new_row

In [16]:
def get_rarity_columns(data, new_row):
    new_row['rarity_common'] = int(data['rarity'] == 'common')
    new_row['rarity_uncommon'] = int(data['rarity'] == 'uncommon')
    new_row['rarity_rare'] = int(data['rarity'] == 'rare')
    new_row['rarity_mythic'] = int(data['rarity'] == 'mythic')
    new_row['rarity_multiclass'] = data['rarity']
    
    return new_row

In [17]:
def get_prices_columns(data, new_row):
    has_price = data.get('prices', False) != False
    if has_price:
        has_price_eur = ('eur' in data['prices']) and (data['prices']['eur'] is not None)
        if has_price_eur:
            new_row['price_eur'] = float(data['prices']['eur'])
        else:
            new_row['price_eur'] = np.NaN
    else:
        new_row['price_eur'] = np.NaN
    
    new_row['price_cheap'] = int(new_row['price_eur'] <= 0.15)
    new_row['price_moderate'] = int(new_row['price_eur'] > 0.15 and new_row['price_eur'] <= 3.0)
    new_row['price_expensive'] = int(new_row['price_eur'] > 3.0)
    
    if new_row['price_eur'] <= 0.15:
        new_row['price_multiclass'] = 'cheap'
    elif new_row['price_eur'] <= 3.0:
        new_row['price_multiclass'] = 'moderate'
    else:
        new_row['price_multiclass'] = 'expensive'
    
    return new_row

In [18]:
def get_legalities_columns(data, new_row): 
    for key in data['legalities']:
        new_row[key + '_legal'] = int(data['legalities'][key] == 'legal')
        new_row[key + '_not_legal'] = int(data['legalities'][key] == 'not_legal')
        new_row[key + '_restricted'] = int(data['legalities'][key] == 'restricted')
        new_row[key + '_banned'] = int(data['legalities'][key] == 'banned')
    
    return new_row

In [19]:
def get_combat_columns(data, new_row):
    has_power = data.get('power', False) != False
    if has_power:
        new_row['power_None'] = 0
        new_row['power_*'] = int(data['power'] == '*')
        new_row['power_1+*'] = int(data['power'] == '1+*')
        new_row['power_2+*'] = int(data['power'] == '2+*')
        for i in range(-1, 17):
            new_row['power_' + str(i)] = int(data['power'] == i)
    else:
        new_row['power_None'] = 1
        new_row['power_*'] = 0
        new_row['power_1+*'] = 0
        new_row['power_2+*'] = 0
        for i in range(-1, 17):
            new_row['power_' + str(i)] = 0
            
    has_toughness = data.get('toughness', False) != False
    if has_toughness:
        new_row['toughness_None'] = 0
        new_row['toughness_*'] = int(data['toughness'] == '*')
        new_row['toughness_*+1'] = int(data['toughness'] == '*+1')
        new_row['toughness_1+*'] = int(data['toughness'] == '1+*')
        new_row['toughness_2+*'] = int(data['toughness'] == '2+*')
        new_row['toughness_7-*'] = int(data['toughness'] == '7-*')
        for i in range(-1, 18):
            new_row['toughness_' + str(i)] = int(data['toughness'] == i)
    else:
        new_row['toughness_None'] = 1
        new_row['toughness_*'] = 0
        new_row['toughness_*+1'] = 0
        new_row['toughness_1+*'] = 0
        new_row['toughness_2+*'] = 0
        new_row['toughness_7-*'] = 0
        for i in range(-1, 18):
            new_row['toughness_' + str(i)] = 0

    has_loyalty = data.get('loyalty', False) != False
    if has_loyalty:
        new_row['loyalty_None'] = 0
        new_row['loyalty_X'] = int(data['loyalty'] == 'X')
        for i in range(1, 8):
            new_row['loyalty_' + str(i)] = int(data['loyalty'] == i)
    else:
        new_row['loyalty_None'] = 1
        new_row['loyalty_X'] = 0
        for i in range(1, 8):
            new_row['loyalty_' + str(i)] = 0       
            
    return new_row

In [20]:
def get_keywords_columns(data, new_row):
    
    total_keywords = 0
    
    for keyword in keyword_actions:
        keyword_present = int(keyword in data['keywords'])
        new_row['keyword_' + keyword.lower().replace(' ', '_')] = keyword_present
        total_keywords += keyword_present
        
    for keyword in keyword_abilities:
        keyword_present = int(keyword in data['keywords'])
        new_row['keyword_' + keyword.lower().replace(' ', '_')] = keyword_present
        total_keywords += keyword_present
    
    new_row['total_keywords'] = total_keywords
    
    return new_row

In [21]:
def get_custom_categories_columns(data, new_row):
    rules_text = ""
    
    has_text = data.get('oracle_text', False) != False
    if has_text:
        rules_text += str(data['oracle_text'])
        rules_text = rules_text.replace(str(data['name']), 'CARD_NAME')
        
    is_dfc = data.get('card_faces', False) != False
    if is_dfc:
        for face in data['card_faces']:
            face_data = json.loads(json.dumps(face, indent = 4))
            face_has_column = face_data.get('oracle_text', False) != False
            if face_has_column:
                rules_text += " " + str(face_data['oracle_text'])
                rules_text = rules_text.replace(str(face_data['name']), 'CARD_NAME')
                
    type_line = str(data['type_line']).lower()
    
    #counterspell
    new_row['custom_' + 'counterspell'] = 1 if re.search("((?<![Ww]ould)(?<![Ii]f).*[Cc]ounter[^.,]*spell.*[.])|((each|all|any)[^.]*,[^.]*counter)|([Ee]xile[^.,]*spells)", rules_text) else 0
    if (('flash' not in data['keywords']) and ("instant" not in type_line) and ("sorcery" not in type_line)): new_row['custom_' + 'counterspell'] = 0
        
    #manarock
    new_row['custom_' + 'manarock'] = 1 if data.get('produced_mana', False) != False else 0
    if (re.search("(?<![Tt]his spell )(costs?)[^.,]*less to (cast|activate)", rules_text)): new_row['custom_' + 'manarock'] = 1
    if (("artifact" not in type_line) or ("creature" in type_line)): new_row['custom_' + 'manarock'] = 0
        
    #manadork
    new_row['custom_' + 'manadork'] = 1 if data.get('produced_mana', False) != False else 0
    if ("creature" not in type_line): new_row['custom_' + 'manadork'] = 0
        
    #point_removal
    new_row['custom_' + 'point_removal'] = 1 if re.search("((?![^.,]*can't)[^.,]*[Dd]estroy[^.,]*target)|([Ee]xile[^.,]*target(?![^.,]*graveyard)(?![^.,]*hand)(?![^.,]*player)(?![^.,]*you control).*[.])|((opponent|([Ee]ach|[Tt]arget) player)(?!can|may)[^,.]*sacrifices?(?!d)(?=[^.,]*( a |target)))|([Tt]arget creature[^,.]*\/[-]\d)|(deal[^,.]*damage[^,.]*(any target|target creature))|([Pp]ut target(?![^.,]*graveyard)(?![^.,]*hand)[^,.]*bottom[^,.]*library)", rules_text) else 0
        
    #exile
    new_row['custom_' + 'exile'] = 1 if re.search("([Ee]xile)|([Rr]emove[^,.]*from the game)", rules_text) else 0
    
    #boardwipe
    new_row['custom_' + 'boardwipe'] = 1 if re.search("((([Dd]estroy|[Ee]xile|[Ss]acrifice) (all|each)(?![^,.]*from)(?![^,.]*chosen)(?![^,.]*card)(?![^,.]*spell)(?![^,.]*graveyard)(?![^,.]*hand))|([Rr]eturn (all|each)(?![^,.]*from)(?![^,.]*chosen)(?![^,.]*card)[^,.]*hand))|(([Ee]ach|[Aa]ll)[^,.]*creature[^,.]*-\d*\/-\d*)", rules_text) else 0
    
    #land_ramp
    new_row['custom_' + 'land_ramp'] = 1 if re.search("(((([Bb]asic|[Ff]orest|[Mm]ountain|[Ss]wamp|[Pp]lains|[Ii]sland)[^,.]*card)|land)[^.]*put[^,.]*onto[^,.]*battlefield)|(may play[^,.]*(additional|any number)[^,.]*land)", rules_text) else 0
    
    #nonland_tutor
    new_row['custom_' + 'nonland_tutor'] = 1 if re.search("([Ss]earch(?![^,.]*[Oo]pponent's)[^,.]*library(?![^,.]*[Bb]asic)(?![^,.]*[Ll]and)(?![^,.]*[Mm]ountain)(?![^,.]*[Ff]orest)(?![^,.]*[Pp]lains)(?![^,.]*[Ii]sland)(?![^,.]*[Ss]wamp))", rules_text) else 0
    
    #card_draw
    new_row['custom_' + 'card_draw'] = 1 if re.search("((^|[.])(?![Ww]henever[^,.]*)[Dd]raw[^,.]*card)|(([Ww]henever|[Ii]f)[^.]*,[^.]*[Dd]raw[^,.]*card)", rules_text) else 0
    
    #burn
    new_row['custom_' + 'burn'] = 1 if re.search("([Dd]eal(?![^,.]*combat)[^,.]*damage(?![^,.]*to creature)[^,.]*(player|any target))", rules_text) else 0
    
    #discard
    new_row['custom_' + 'discard'] = 1 if re.search("((^|[.])(?![Ww]henever[^,.]*)[Dd]iscard[^,.]*card)|(([Ww]henever|[Ii]f)[^.]*,[^.]*[Dd]iscard[^,.]*card)", rules_text) else 0
    
    #etb
    new_row['custom_' + 'etb'] = 1 if re.search("(([Ww]hen|[Ww]henever) CARD_NAME enters? the battlefield)", rules_text) else 0
    
    #die_trigger
    new_row['custom_' + 'die_trigger'] = 1 if re.search("(([Ww]hen|[Ww]henever) (CARD_NAME|this (creature|permanent|card)) (dies?|leaves? the battlefield?))", rules_text) else 0
    
    #attack_trigger
    new_row['custom_' + 'attack_trigger'] = 1 if re.search("(([Ww]hen|[Ww]henever) (CARD_NAME|this (creature|permanent|card)) attack)", rules_text) else 0
    
    #tokens
    new_row['custom_' + 'tokens'] = 1 if re.search("(([Cc]reate|[Pp]ut|[Bb]ecome)[^,.]*token)", rules_text) else 0
    
    #creature_tokens
    new_row['custom_' + 'creature_tokens'] = 1 if re.search("(([Cc]reate|[Pp]ut|[Bb]ecome)[^,.]*creatures? token)", rules_text) else 0
    
    #treasure_tokens
    new_row['custom_' + 'treasure_tokens'] = 1 if re.search("(([Cc]reate|[Pp]ut|[Bb]ecome)[^,.]*[Tt]reasures? token)", rules_text) else 0
    
    #extra_turn
    new_row['custom_' + 'extra_turn'] = 1 if re.search("([Tt]ake[^,.]*extra turn)", rules_text) else 0
    
    #plus1_counters
    new_row['custom_' + 'plus1_counters'] = 1 if re.search("([+]1\/[+]1 counters?)", rules_text) else 0
    
    #graveyard_hate
    new_row['custom_' + 'graveyard_hate'] = 1 if re.search("([Ee]xile[^,.]*(?<!your )graveyard)|([Ss]huffle[^,.]*(?<!your )graveyard[^,.]*into)", rules_text) else 0
    
    #free_spell
    new_row['custom_' + 'free_spell'] = 1 if re.search("((rather than pay|instead of paying) (this spell's|CARD_NAME's) mana cost)", rules_text) else 0
    if (data['cmc'] == 0 and ("Suspend" not in data['keywords'])): new_row['custom_' + 'free_spell'] = 1
    
    #bounce_spell
    new_row['custom_' + 'bounce_spell'] = 1 if re.search("([Rr]eturn (target|all|each|chosen)(?![^,.]*from)(?![^,.]*card)[^,.]*hand)", rules_text) else 0
    
    #sac_outlet
    new_row['custom_' + 'sac_outlet'] = 1 if re.search("([Ss]acrifice[^,.:]*(?<!this )(creature|enchantment|artifact|land|permanent|planeswaker)[^.:]*:)", rules_text) else 0
    
    #sac_payoff
    new_row['custom_' + 'sac_payoff'] = 1 if re.search("(([Ww]hen|[Ww]henever) (?!CARD_NAME)(?!this)(?!enchanted)[^,.]*(?<!CARD_NAME) (dies?|leaves? the battlefield|sacrificed))|(([Ww]hen|[Ww]henever) CARD_NAME or another [^,.]* (dies?|leaves? the battlefield|sacrificed))", rules_text) else 0
    
    #cant_counter
    new_row['custom_' + 'cant_counter'] = 1 if re.search("([Cc]an't be countered)", rules_text) else 0
    
    #tax
    new_row['custom_' + 'tax'] = 1 if re.search("(unless (?!you)[^,.]*pay)|(more to cast)", rules_text) else 0
    
    #graveyard_to_hand
    new_row['custom_' + 'graveyard_to_hand'] = 1 if re.search("([Rr]eturn[^,.]*card from[^,.]*graveyard[^,.]*hand)", rules_text) else 0
    
    #reanimation
    new_row['custom_' + 'reanimation'] = 1 if re.search("([Rr]eturn[^,.]*card[^,.]*from[^,.]*graveyard[^,.]*battlefield)|([Cc]hoose[^,.]*card[^,.]*graveyard.*[Rr]eturn[^,.]*(it|them)[^,.]*battlefield)", rules_text) else 0
    
    #cast_graveyard
    new_row['custom_' + 'cast_graveyard'] = 1 if re.search("([Cc]ast[^,.]*from[^,.]*graveyard)", rules_text) else 0
    
    #lord
    new_row['custom_' + 'lord'] = 1 if re.search("((?<!lose )([Ee]ach|[Aa]ll(?!y)|(?<![Aa]n)[Oo]ther)[^,.]*(get[^,.]*[+]\d+\/[+]\d+|gain(?![^,.]*life)|have|has))", rules_text) else 0
    if ("creature" not in type_line): new_row['custom_' + 'lord'] = 0
    
    #upkeep_trigger
    new_row['custom_' + 'upkeep_trigger'] = 1 if re.search("([Aa]t the beginning of[^,.]*upkeep)", rules_text) else 0
    
    #endstep_trigger
    new_row['custom_' + 'endstep_trigger'] = 1 if re.search("([Aa]t the beginning of[^,.]*end step)", rules_text) else 0
    
    #combat_trigger
    new_row['custom_' + 'combat_trigger'] = 1 if re.search("([Aa]t the beginning of[^,.]*(?<!pre)combat)", rules_text) else 0
    
    #life_gain
    new_row['custom_' + 'life_gain'] = 1 if re.search("([Gg]ain[^,.]life)|(life total becomes)", rules_text) else 0
    
    #cost_reduction
    new_row['custom_' + 'cost_reduction'] = 1 if re.search("(?<![Tt]his spell )(costs?)[^.,]*less to (cast|activate)", rules_text) else 0
    
    #mana_multipliers
    new_row['custom_' + 'mana_multipliers'] = 1 if re.search("(([Dd]ouble|[Tt]riple)[^,.]*mana)|(([Ww]henever|[Ww]hen)[^,.]*tap[^,.]*mana[^.]*(add[^,.]*mana|mana[^,.]*instead|additional))", rules_text) else 0
    
    #card_selection
    new_row['custom_' + 'card_selection'] = 1 if re.search("([Dd]raw[^,.]*cards[^,.]*, then[^,.]*(put|return)[^,.]*(library|exile|graveyard))|(([Ll]ook|[Rr]eveal)[^,.]*cards[^,.]*library.*[Pp]ut[^,.]*hand)|([Ss]cry \d+)", rules_text) else 0
    
    #cast_trigger
    new_row['custom_' + 'cast_trigger'] = 1 if re.search("(([Ww]hen|[Ww]henever)[^,.]*(cast|play(?!er)))", rules_text) else 0
    
    #gain_control
    new_row['custom_' + 'gain_control'] = 1 if re.search("([Gg]ain[^,.]*control)", rules_text) else 0
    
    #evasion
    new_row['custom_' + 'evasion'] = 1 if re.search("([Uu]nblockable)|([Cc]an be blocked only by)|(([Cc]an't|[Cc]an not) be blocked)", rules_text) else 0
    
    #copy
    new_row['custom_' + 'copy'] = 1 if re.search("([Cc]opy )", rules_text) else 0
    
    #trigger_multiplier (???)
    new_row['custom_' + 'trigger_multiplier'] = 1 if re.search("([Tt]rigger[^,.]*additional)|([Cc]opy[^,.]*trigger)", rules_text) else 0
    
    #damage_multipliers
    new_row['custom_' + 'damage_multipliers'] = 1 if re.search("(([Ii]f|[Ww]hen|[Ww]henever)[^,.]*source[^,.]*deal[^,.]*damage[^.]*damage[^,.]*instead)", rules_text) else 0
    
    #instant_speed_enabler
    new_row['custom_' + 'instant_speed_enabler'] = 1 if re.search("(as though (it|they) had flash)", rules_text) else 0
    
    #doublers
    new_row['custom_' + 'doublers'] = 1 if re.search("([Dd]ouble|[Tt]riple|[Tt]wice)|(for each[^.]*(create|put))", rules_text) else 0
    
    #blinker
    new_row['custom_' + 'blinker'] = 1 if re.search("([Ee]xile.*[Rr]eturn[^,.]*battlefield[^,.]*control)", rules_text) else 0
    
    #play_toplibrary
    new_row['custom_' + 'play_toplibrary'] = 1 if re.search("(may (play|cast)[^,.]*from[^,.]*top[^,.]*library)", rules_text) else 0
    
    #life_lose
    new_row['custom_' + 'life_lose'] = 1 if re.search("([Ll]ose[^,.]*life)", rules_text) else 0
    
    #disenchant
    new_row['custom_' + 'disenchant'] = 1 if re.search("(([Dd]estroy|[Ee]xile)[^,.]*((?<!non)enchantment|(?<!non)artifact))|(([Ss]huffle|[Pp]ut)(?![^,.]*counter)[^,.]*((?<!non)enchantment|(?<!non)artifact)(?![^,.]*battlefield))", rules_text) else 0
    
    #animator
    new_row['custom_' + 'animator'] = 1 if re.search("([Bb]ecome[^,.]*creature)|((is|are|as)[^,.]*creature[^,.]*in addition[^,.]*other types)", rules_text) else 0
    
    #wish
    new_row['custom_' + 'wish'] = 1 if re.search("(you own from outside the game)", rules_text) else 0
    
    #cheatinto_play
    new_row['custom_' + 'cheatinto_play'] = 1 if re.search("([Pp]ut[^,.]*card[^,.]*hand[^,.]*(battlefield|into play))|([Cc]ast[^,.]*without[^,.]*pay[^,.]*mana)", rules_text) else 0
    
    #ritual
    new_row['custom_' + 'ritual'] = 1 if data.get('produced_mana', False) != False else 0
    if (("instant" not in type_line) or ("sorcery" in type_line)): new_row['custom_' + 'ritual'] = 0
    
    #no_maximum
    new_row['custom_' + 'no_maximum'] = 1 if re.search("(no maximum hand size)", rules_text) else 0
    
    #wheel
    new_row['custom_' + 'wheel'] = 1 if re.search("([Dd]iscard[^,.]*hand.*[Dd]raw[^,.]*(that many|equal to|seven|7))", rules_text) else 0
    
    #extra_combat
    new_row['custom_' + 'extra_combat'] = 1 if re.search("(additional combat phase)", rules_text) else 0
    
    #pillow_fort
    new_row['custom_' + 'pillow_fort'] = 1 if re.search("(([Ww]hen|[Ww]henever)[^,.]*((attack[^,.]*you(?!r))|(deal[^,.]*damage[^,.]*to you(?!r)))[^,.]*)|([Cc]reatures[^,.]*(can not|can't)[^,.]*attack)|([Pp]revent[^,.]*damage[^,.]*to you(?!r))|([Dd]eal[^,.]*no[^,.]*combat damage)", rules_text) else 0
    
    #land_destruction
    new_row['custom_' + 'land_destruction'] = 1 if re.search("(([Dd]estroy|[Ee]xile)[^,.:]*((?<!non)land|(?<!nonland )permanent))", rules_text) else 0
    
    #win_game
    new_row['custom_' + 'win_game'] = 1 if re.search("((?<!can't )(?<!can not )win the game)|(can't lose the game)", rules_text) else 0
    
    #lose_game
    new_row['custom_' + 'lose_game'] = 1 if re.search("((?<!can't )(?<!can not )lose the game)|(can't win the game)", rules_text) else 0
    
    #poison
    new_row['custom_' + 'poison'] = 1 if re.search("(poison counter)", rules_text) else 0
    
    #new_row['custom_' + ''] = 1 if re.search("", rules_text) else 0
    
    return new_row

In [22]:
def generate_prints_columns(df):
    df.insert(3, "prints", 0)
    df['prints'] = df.groupby('name').prints.transform('count')

In [23]:
def fix_missing_prices_columns(df):
    df['price_eur'] = df.apply(
        lambda row: (df.loc[df['name'] == row['name']])['price_eur'].median() if np.isnan(row['price_eur']) else row['price_eur'],
        axis=1
    )
    df['price_eur'].fillna(0.0, inplace=True)

# -------------------- FEATURE EXTRACTION --------------------

In [24]:
# opening the JSON file
file = open('API\\' + filename + '.json', 'r', encoding="utf8")

In [25]:
df = pd.DataFrame()

In [26]:
count = 0
#while count <= 5000:
while True:
    count += 1
    #print(count)
  
    # Get next line from file
    line = file.readline()
    
    # if line is empty
    # end of file is reached
    if not line:
        break
    
    # Remove enter sign
    line = line[:-1]
    
    # Dont analize lines that do not containg info
    if len(line) <= 1:
        continue
        
    # Remove ',' sign if it is not a last line       
    if line[-1] == ',':
        line = line[:-1]
   
    # Reading the JSON
    data = json.loads(line)
    
    # Check if card should be skipped
    if check_if_should_be_skipped(data):
        continue
    
    new_row = {}
    new_row = get_basic_columns(data, new_row)
    new_row = get_types_columns(data, new_row)
    new_row = get_mana_columns(data, new_row)
    new_row = get_rarity_columns(data, new_row)
    new_row = get_prices_columns(data, new_row)
    new_row = get_legalities_columns(data, new_row)
    new_row = get_combat_columns(data, new_row)
    new_row = get_keywords_columns(data, new_row)
    new_row = get_custom_categories_columns(data, new_row)
    
    df = df.append(new_row, ignore_index = True)

In [27]:
# Closing file
file.close()

In [28]:
generate_prints_columns(df)
fix_missing_prices_columns(df)

In [29]:
df.head()

,name,set,dfc,prints,layout,flavor_text,type_creature,type_artifact,type_enchantment,type_planeswalker,...,custom_cheatinto_play,custom_ritual,custom_no_maximum,custom_wheel,custom_extra_combat,custom_pillow_fort,custom_land_destruction,custom_win_game,custom_lose_game,custom_poison
0,Fury Sliver,tsp,0.0,3,normal,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Kor Outfitter,zen,0.0,2,normal,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Siren Lookout,xln,0.0,1,normal,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Venerable Knight,eld,0.0,1,normal,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Wildcall,ugin,0.0,2,normal,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# saving result to .csv file
df.to_csv(result_file_name + ".csv", index=False, sep=';')

In [31]:
df

,name,set,dfc,prints,layout,flavor_text,type_creature,type_artifact,type_enchantment,type_planeswalker,...,custom_cheatinto_play,custom_ritual,custom_no_maximum,custom_wheel,custom_extra_combat,custom_pillow_fort,custom_land_destruction,custom_win_game,custom_lose_game,custom_poison
0,Fury Sliver,tsp,0.0,3,normal,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Kor Outfitter,zen,0.0,2,normal,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Siren Lookout,xln,0.0,1,normal,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Venerable Knight,eld,0.0,1,normal,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Wildcall,ugin,0.0,2,normal,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36254,Tezzeret's Gambit,nph,0.0,7,normal,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36255,Morkrut Banshee,isd,0.0,4,normal,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36256,Angel's Tomb,ori,0.0,2,normal,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36257,Horned Troll,8ed,0.0,1,normal,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
